# Basic training of YOLO v11 detection model

**Libraries installation and import**

In [ ]:
!pip install ultralytics

In [ ]:
from google.colab.patches import cv2_imshow
from PIL import ImageFont, ImageDraw, Image
from ultralytics import YOLO
from pathlib import Path
import numpy as np
import kagglehub
import shutil
import random
import yaml
import cv2
import os

---

**Dataset download**

In [ ]:
# download dataset from kaggle
path = kagglehub.dataset_download("nguyngiabol/colorful-fashion-dataset-for-object-detection")
print("Path to dataset files:", path)

In [ ]:
# create directories for dataset store
base_dir = Path("/content/colorful_object_detection/")
image_dir = base_dir / "images"
label_dir = base_dir / "labels"

for folder in [image_dir, label_dir]:
  folder.mkdir(parents=True, exist_ok=True)

In [ ]:
# copy data to the current directory 'colorful_object_detection'
source_root = Path('/kaggle/input/colorful-fashion-dataset-for-object-detection/colorful_fashion_dataset_for_object_detection/')
source_imgs = source_root / 'JPEGImages'
source_annots = source_root / 'Annotations_txt'
for img in source_imgs.glob('*'):
    shutil.copy(img, image_dir / img.name)
    annot = source_annots / f'{img.stem}.txt'
    shutil.copy(annot, label_dir / annot.name)
print("Copy is done")

In [ ]:
# directories creation
for split in ['train', 'val']:
  (image_dir / split).mkdir(parents=True, exist_ok=True)
  (label_dir / split).mkdir(parents=True, exist_ok=True)

# files shuffle
files = list(image_dir.glob("*.jpg"))
np.random.shuffle(files)

# split on train/val
train_cnt = int(len(files) * 0.9)
train_files, val_files = files[:train_cnt], files[train_cnt:]

# move files to train/val directories
for index, img_path in enumerate(files):
    split = 'train' if index < train_cnt else 'val'

    dest_img = image_dir / split / img_path.name
    shutil.move(str(img_path), str(dest_img))

    txt_path = label_dir / (img_path.stem + '.txt')
    dest_txt = label_dir / split / txt_path.name
    shutil.move(str(txt_path), str(dest_txt))

---

**Data Overview**

In [ ]:
img = cv2.imread(str(image_dir / 'train' / '100599.jpg'))
print(f"Shape of the image: {img.shape}")
cv2_imshow(img)

In [ ]:
labels = []
with open(label_dir / 'train' / '100599.txt') as f:
    labels = [line.strip() for line in f if line.strip()]
print("Clothes' bounding boxes in an image:")
for label in labels:
    cls, xmin, ymin, xmax, ymax = label.split()
    print(f"Class {cls}, xmin: {xmin}, ymin: {ymin}, xmax: {xmax}, ymax: {ymax}")

In [ ]:
def get_annotation(img, labels):
  img_height, img_width, img_channels = img.shape
  for obj in labels:
    name, x, y, width, height = obj.split(" ")
    x, y, width, height = map(float, [x, y, width, height])
    x1_real = int(np.abs(x - (width/2)) * img_width)
    x2_real = int(np.abs(x + (width/2)) * img_width)
    y1_real = int(np.abs(y + (height/2)) * img_height)
    y2_real = int(np.abs(y - (height/2)) * img_height)
    color = (0, 0, 0)
    image = cv2.rectangle(img, (x1_real, y2_real), (x2_real, y1_real), color = color, thickness = 2)
    image = cv2.putText(img, name, (x1_real, y2_real), fontFace=cv2.FONT_HERSHEY_DUPLEX,
                      fontScale=1.0, color=color, thickness=2)
  return img

img = get_annotation(img, labels)
cv2_imshow(img)

---

**Compose metadata about dataset in yaml file**

In [ ]:
data = {
    'nc': 10,
    'train': 'colorful_object_detection/images/train',
    'val': 'colorful_object_detection/images/val',
    'names': {
        0: 'sunglass',
        1: 'hat',
        2: 'jacket',
        3: 'shirt',
        4: 'pants',
        5: 'shorts',
        6: 'skirt',
        7: 'dress',
        8: 'bag',
        9: 'shoe'
    }
}

with open('data.yaml', 'w') as file:
  yaml.dump(data, file)

---

**YOLO v11 Model Fine-Tuning**

In [ ]:
model = YOLO("yolo11m.pt")

results = model.train(
    data='data.yaml',
    epochs=100,
    batch=64,
    device=[0],
    freeze=8,
    lr0=3e-4,
    augment=True,
    lrf=0.1
)